In [ ]:
#Regression form used Y=X*B (where B includes the intercept ) 
#B found out using B=((X'X)^-1)*X'*Y
#X is row vectors of the form [RSSI_1,RSSI_2,RSSI_3,RSSI_3,RSSI_4,1]
#Y is a row vectors of log of the distances from NUCs of the form [dist_1,dist_2,dist_3,dist_4]


In [1]:
import pandas as pd
import numpy as np
import math
from sqlalchemy import create_engine
def calc_dist(x1,y1,x2,y2):
  distance=math.sqrt((x1-x2)**2 + (y1-y2)**2 )
  return distance
eng= create_engine('sqlite:///new_parkinglot.db')
df=pd.read_sql_table('training',eng)
#print(calc_dist(1,2,3,4))
a=df['NUC1']
b=df['NUC2']
c=df['NUC3']
d=df['NUC4']
x=df['LOCX']
y=df['LOCY']
#print(a)
NUC1_XY=[0,0]
NUC2_XY=[0,-10]
NUC3_XY=[10,-10]
NUC4_XY=[10,0]
X=[a[0],b[0],c[0],d[0],1]

for index in range(1,a.size-1):
  X =np.vstack((X,[a[index],b[index],c[index],d[index],1]))
#print(X.shape)
distance1 = calc_dist(NUC1_XY[0], NUC1_XY[1], x[0], y[0])
distance2 = calc_dist(NUC2_XY[0], NUC2_XY[1], x[0], y[0])
distance3 = calc_dist(NUC3_XY[0], NUC3_XY[1], x[0], y[0])
distance4 = calc_dist(NUC4_XY[0], NUC4_XY[1], x[0], y[0])
Y=[distance1,distance2,distance3,distance4]
for index in range(1,a.size-1):
  distance1=calc_dist(NUC1_XY[0],NUC1_XY[1],x[index],y[index])
  distance2 = calc_dist(NUC2_XY[0], NUC2_XY[1], x[index], y[index])
  distance3 = calc_dist(NUC3_XY[0], NUC3_XY[1], x[index], y[index])
  distanc4 = calc_dist(NUC4_XY[0], NUC4_XY[1], x[index], y[index])
  Y=np.vstack((Y,[math.log10(distance1),math.log10(distance2),math.log10(distance3),math.log10(distance4)]))
#print(Y.shape)
temp1=np.matmul(np.matrix.transpose(X),X)
#print(temp1.shape)
temp2=np.matmul(np.linalg.inv(temp1),np.matrix.transpose(X))
#beta_hat is the parameter used for regression
beta_hat=np.matmul(temp2,Y)
#print(beta_hat.shape)
#using the model on test data
df2=pd.read_sql_table('test',eng)
a=df2['NUC1']
b=df2['NUC2']
c=df2['NUC3']
d=df2['NUC4']
#print(a.size)
X_in=[a[0],b[0],c[0],d[0],1]
for index in range(1,a.size-1):
  X_in =np.vstack((X_in,[a[index],b[index],c[index],d[index],1]))
Y_out=np.matmul(X_in,beta_hat)
Y_test=10**Y_out
NUC1_dist=Y_test[:,0]
NUC2_dist=Y_test[:,1]
NUC3_dist=Y_test[:,2]
NUC4_dist=Y_test[:,3]
output=pd.DataFrame({'NUC1':NUC1_dist,'NUC2':NUC2_dist,'NUC3':NUC3_dist,'NUC4':NUC4_dist})
output.to_csv("test_output_richarddoss.csv")

#Y=[np.ndarray([y0, y1]) for y0, y1 in zip(x,y)]